In [22]:
### parameters to get extinction coeff for NPA at give pH value
pH = 7.0
e_NPA = 18300  ### extinction coeff for NPA anion
pKa_NPA = 7.15 ### pKa for p-nitrophenol

### List of kcat values for the enzyme in each lane (s^-1)
kcat_list = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100 ]
kcat_list = np.array(kcat_list)
kcat_list = kcat_list * 60    ### convert from s^-1 to min^-1

### List of KM values for the enzyme in each lane (mM)
### Note: Values must not be zero. For no enzyme, use zero concentration in next list
KM_list = [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
KM_list = np.array(KM_list)
KM_list = KM_list * 1E-3       ### convert from mM to M

### List of the enzyme concentrations in each lane (nanomolar)
Max_E_conc = 1   ### nanomolar
E_conc_list = [0, 0, 0, 1, 1, 1, 1/2, 1/2, 1/2, 1/4, 1/4, 1/4]
E_conc_list = np.array(E_conc_list)  * Max_E_conc
E_conc_list = E_conc_list * 1E-9  ### convert from nM to M

### List of Lane Names
lane_name_list = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]

### List of Row Names
row_name_list = ["1", "2", "3", "4", "5", "6", "7", "8"]

### List of the NPA conc in the eight wells of each lane (mM)
S_conc_list = [0.01, 0.02, 0.03, 0.04, 0.06, 0.07, 0.1, 0.2]
S_conc_list = np.array(S_conc_list)
S_conc_list = S_conc_list * 1E-3       ### convert from mM to M

### Calculated Values from the above lists
Vmax_list = kcat_list * E_conc_list   ### Vmax from kcat and [E]

Ka = 10 ** -pKa_NPA   ### extinction coeff for NPA at given pH
H = 10 ** -pH
e_NPA = e_NPA * (Ka / (H + Ka))


In [23]:
### This creates the data set separate x,y data files names as lane,row

data_name = "data1/data"
data_file_extention = ".csv"

import numpy as np
import pandas as pd

eq,f = get_integrated_MM_function()

time_start = 0.5
time_end = 60           ### The end time
n_points = 360          ### number of points - increase if needed

voltage_error = 0.001   ### parameters to define output range and error
random_error = 0.005
max_value = 4

dt = time_end / n_points          ### time step, delta t
t_line = np.arange(time_start,    ### time vector (list of time points)
                   time_end + dt, 
                   dt) 

### Note: Lane names, enzyme conc list, KM list and Vmax list must all be
### same length or this will fail. Row names and row concentration lists 
### must also have equal lengths.

parameters = zip(lane_name_list, Vmax_list, KM_list, E_conc_list)
row_info = zip(row_name_list, S_conc_list)
for p in parameters:
    lane_name, Vmax_value, KM_value, E_conc = p   ### unpack kcat, KM and [E]

    for row in row_info:
        row_name, S0_value = row      ### unpack row name and substrate conc
        plate_df = pd.DataFrame([])   ### start with empty dataframe

        ### Calculate product from enzyme reaction 
        product_E = S0_value - f(t_line, S0_value, KM_value, Vmax_value)   
        product_E = np.real(product_E)  ### complex numbers fixed

        ### Calculate product from uncatalyzed reaction 
        product_NPA = S0_value - S0_value * np.exp(-1E-3 * t_line)
        product = product_E + product_NPA
        absorbance = product * e_NPA   ### result in absorbance units

        ### Add voltage error 
        fraction_transmittance  = 1 / (10 ** absorbance)                      
        fraction_transmittance = np.random.normal(fraction_transmittance, 
                                                  voltage_error, 
                                                  len(fraction_transmittance))
        absorbance = -np.log10(fraction_transmittance)

        ### Add random error
        absorbance = np.random.normal(absorbance,     
                                      random_error, 
                                      len(absorbance))   

        absorbance[absorbance > max_value] = max_value   ### cap values 
        absorbance = np.nan_to_num(absorbance,  ### replace NaN with max value
                          copy = True, 
                          nan = max_value)   

        x = t_line; y = absorbance
        plate_df["time"]=x
        plate_df["abs"]=y

        
        out_file_name = data_name + "_" + lane_name + "_" \
            + row_name + data_file_extention
        plate_df.to_csv(out_file_name)



1e-05
1
2e-05
2
3e-05
3
4e-05
4
6e-05
5
7.000000000000001e-05
6
0.0001
7
0.0002
8


In [15]:
#def integrated_result_at_time:

def get_integrated_MM_function():
    """Returns the integrated Michaelis-Meneten rate law and a function
    
    No parameters
    
    returns (eq, f):
        eq (SymPy equation object): 
            The integrated MM rate law
        f (function object): f(t, S0, KM, Vmax)
            t (float or array): time point or array of points
            S0 (float): Initial substrate concentration
            KM (float): KM value for enzyme
            Vmax (float): Vmax for enzyme
            returns s (float or array): substrate conc at time = t
    
    Example: 
    MMeq, s_conc = get_integrated_MM_function(t, S0, KM, Vmax)
    display(MMeq) # show the integrated rate equation
    s = s_conc(t_list, S0_value, KM_value, Vmax_value) # s at each t in t_list

    """
    #########################################
    ### Stolen code to created analytically integrated function for S vs, t
    #########################################

    import sympy as sym

    ##########################
    ### Set up equation 
    ##########################

    t = sym.symbols('t')           ### create x as a 'symbol', not a variable
    Vmax = sym.symbols('V_{max}')  ### create k as a 'symbol'
    St = sym.symbols('S_t')        ### create At as a 'symbol'
    S0 = sym.symbols('S_0')        ### create A0 as a 'symbol'
    KM = sym.symbols('K_M')

    xt = sym.Function('x_t')       ### create x as a 'function', not a variable

    lhs = sym.Derivative(-St, t)   ### Using Derivative function to get differential of A(t) w.r.t. t
    rhs = Vmax*(St/(KM+St))

    diffeq = sym.Eq(lhs, rhs)                   ### create a sympy equation
    diffeq = diffeq.subs({St: (S0 - xt(t))})    ### Substitute the term with S0-x

    ##########################
    ### Solve the differential equation 
    ##########################

    res = sym.dsolve(diffeq, ics={xt(0): 0})    ### Solve the differential equation. Initial condition is x(t) = 0 when t = 0

    ##########################
    ### Clean up algebra 
    ##########################

    eq = res.subs(xt(t), S0-St)            ### substitute x for So - St
    eq = sym.simplify(eq)                  ### Simplify the result
    eq = sym.Eq(eq.lhs - S0, eq.rhs - S0)  ### Subtract S0 from both sides of the equation
    eq = sym.Eq(-eq.lhs, -eq.rhs)          ### take the negative of both sides of the equation 

    ##########################
    ### Display the final form of equation 
    ##########################

    #print("The integrated rate law for the MM equation")
#    display(eq)                         

    ##########################
    ### create function in terms of t, S0, KM and Vmax
    ##########################

    f = sym.lambdify([t, S0, KM, Vmax], eq.rhs)   
    return(eq,f)

eq,q = get_integrated_MM_function()
help(get_integrated_MM_function)

Help on function get_integrated_MM_function in module __main__:

get_integrated_MM_function()
    Returns the integrated Michaelis-Meneten rate law and a function
    
    No parameters
    
    returns (eq, f):
        eq (SymPy equation object): 
            The integrated MM rate law
        f (function object): f(t, S0, KM, Vmax)
            t (float or array): time point or array of points
            S0 (float): Initial substrate concentration
            KM (float): KM value for enzyme
            Vmax (float): Vmax for enzyme
            returns s (float or array): substrate conc at time = t
    
    Example: 
    MMeq, s_conc = get_integrated_MM_function(t, S0, KM, Vmax)
    display(MMeq) # show the integrated rate equation
    s = s_conc(t_list, S0_value, KM_value, Vmax_value) # s and each t in t_list



In [ ]:
from matplotlib import pyplot as plt
plt.ioff()      ### switch off interactive display of plots. plt.show() needed to display a plot now
